## 6. Nonlinear ensemble filtering for the Lorenz-63 problem

In this notebook, we are interested in the sequential inference 



References: 


[1] Evensen, G., 1994. Sequential data assimilation with a nonlinear quasi‐geostrophic model using Monte Carlo methods to forecast error statistics. Journal of Geophysical Research: Oceans, 99(C5), pp.10143-10162.

[2] Asch, M., Bocquet, M. and Nodet, M., 2016. Data assimilation: methods, algorithms, and applications. Society for Industrial and Applied Mathematics.

[3] Bishop, C.H., Etherton, B.J. and Majumdar, S.J., 2001. Adaptive sampling with the ensemble transform Kalman filter. Part I: Theoretical aspects. Monthly weather review, 129(3), pp.420-436. 

[4] Lorenz, E.N., 1963. Deterministic nonperiodic flow. Journal of atmospheric sciences, 20(2), pp.130-141.

[5] Spantini, A., Baptista, R. and Marzouk, Y., 2019. Coupling techniques for nonlinear ensemble filtering. arXiv preprint arXiv:1907.00389.

### The basic steps
To carry out sequential inference in `AdaptiveTransportMap`, we need to carry out a few basic steps:
* **Specify the problem**: Define the state-space model: initial condition, dynamical and observation models (including process and observation noise)
* **Specify the inflation parameters**: Determine the levels of covariance inflation to properly balance the dynamical system and the observations from the truth system
* **Specify the filter**: Choose the ensemble filter to assimilate the observations in the state estimate
* **Perform the sequential inference**: Perform the sequential inference

We will go through all of these here.

In [3]:
using Revise
using LinearAlgebra
using TransportBasedInference
using Statistics
using Distributions
using OrdinaryDiffEq

Precompiling TransportBasedInference
        Info Given TransportBasedInference was explicitly requested, output will be shown live 
┌ Warning: Type annotations on keyword arguments not currently supported in recipes.
│ Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/BRe07/src/RecipesBase.jl:117
┌ Warning: Type annotations on keyword arguments not currently supported in recipes.
│ Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/BRe07/src/RecipesBase.jl:117
┌ Warning: Type annotations on keyword arguments not currently supported in recipes.
│ Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/BRe07/src/RecipesBase.jl:117
┌ Warning: Type annotations on keyword arguments not currently supported in recipes.
│ Type information has been discarded
└ @ RecipesBase ~/.julia/packages/RecipesBase/BRe07/src/RecipesBase.jl:117
┌ Warning: Type annotations on keyword arguments not currently supported in

In [4]:
using DelimitedFiles

Load some packages to make nice figures

In [5]:
using Plots
default(tickfont = font("CMU Serif", 9), 
        titlefont = font("CMU Serif", 14), 
        guidefont = font("CMU Serif", 12),
        legendfont = font("CMU Serif", 10),
        grid = false)
# pyplot()

using LaTeXStrings
# PyPlot.rc("text", usetex = "true")
# PyPlot.rc("font", family = "CMU Serif")
# gr()
using ColorSchemes

LoadError: ArgumentError: Package Plots not found in current path.
- Run `import Pkg; Pkg.add("Plots")` to install the Plots package.

The Lorenz-63  model is a three dimensional system that models the atmospheric convection [4]. This system is a classical benchmark problem in data assimilation. The state $\boldsymbol{x} = (x_1, x_2, x_3)$ is governed by the following set of ordinary differential equations:

\begin{equation}
\begin{aligned}
&\frac{\mathrm{d} x_1}{\mathrm{d} t}=\sigma(x_2-x_1)\\
&\frac{\mathrm{d} x_2}{\mathrm{d} t}=x_1(\rho-x_2)-x_2\\
&\frac{\mathrm{d} x_3}{\mathrm{d} t}=x_1 x_2-\beta x_3,
\end{aligned}
\end{equation}

where $\sigma = 10, \beta = 8/3, \rho = 28$. For these values, the system is chaotic and behaves like a strange attractor. We integrate this system of ODEs with time step $\Delta t_{dyn} = 0.05$. The state is fully observed $h(t,\boldsymbol{x}) = \boldsymbol{x}$ with $\Delta t_{obs}=0.1$. The initial distribution $\pi_{\mathsf{X}_0}$ is the standard Gaussian. The process noise is Gaussian with zero mean and covariance $10^{-4}\boldsymbol{I}_3$. The measurement noise has a Gaussian distribution with zero mean and covariance $\theta^2\boldsymbol{I}_3$ where $\theta^2 = 4.0$.

### Simple twin-experiment

Define the dimension of the state and observation vectors

In [6]:
Nx = 3
Ny = 3

3

Define the time steps $\Delta t_{dyn}, \Delta t_{obs}$  of the dynamical and observation models. Observations from the truth are assimilated every $\Delta t_{obs}$.

In [7]:
Δtdyn = 0.05
Δtobs = 0.2

0.2

Define the time span of interest

In [8]:
t0 = 0.0
tf = 100.0
Tf = ceil(Int64, (tf-t0)/Δtobs)

500

 Define the distribution for the initial condition $\pi_{\mathsf{X}_0}$

In [9]:
π0 = MvNormal(zeros(Nx), Matrix(1.0*I, Nx, Nx))

FullNormal(
dim: 3
μ: [0.0, 0.0, 0.0]
Σ: [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]
)


We construct the state-space representation `F` of the system composed of the deterministic part of the dynamical and observation models. 

The dynamical model is provided by the right hand side of the ODE to solve. For a system of ODEs, we will prefer an in-place syntax `f(du, u, p, t)`, where `p` are parameters of the model.
We rely on `OrdinaryDiffEq` to integrate the dynamical system with the Tsitouras 5/4 Runge-Kutta method adaptive time marching. 

We assume that the state is fully observable, i.e. $h(x, t) = x$.

In [10]:
h(x, t) = x
F = StateSpace(lorenz63!, h)

StateSpace(TransportBasedInference.lorenz63!, TransportBasedInference.var"#27#28"(), h)

Define the additive inflation for the dynamical and observation models

In [11]:
### Process and observation noise
σx = 1e-1
σy = 2.0

ϵx = AdditiveInflation(Nx, zeros(Nx), σx)
ϵy = AdditiveInflation(Ny, zeros(Ny), σy)

AdditiveInflation(3, [0.0, 0.0, 0.0], [4.0 0.0 0.0; 0.0 4.0 0.0; 0.0 0.0 4.0], [2.0 0.0 0.0; 0.0 2.0 0.0; 0.0 0.0 2.0])

In [12]:
ϵx.Σ

3×3 Diagonal{Float64, Vector{Float64}}:
 0.01   ⋅     ⋅ 
  ⋅    0.01   ⋅ 
  ⋅     ⋅    0.01

In [13]:
model = Model(Nx, Ny, Δtdyn, Δtobs, ϵx, ϵy, π0, 0, 0, 0, F);

To perform the nonlinear ensemble filtering, we first need to estimate the transport map $\boldsymbol{S}^{\boldsymbol{\mathcal{X}}}$.

In this notebook, we are going to assume that the basis of features does not change over time, but solely the coefficients $c_{\boldsymbol{\alpha}}$ of the expansion. 


To estimate the map, we generate joint samples $(\boldsymbol{y}^i, \boldsymbol{x}^i), \; i = 1, \ldots, N_e$ where $\{\boldsymbol{x}^i\}$ are i.i.d. samples from pushforward of the standard Gaussian distribution by the flow of the Lorenz-63 system.

In [14]:
# Time span
tspan = (0.0, 100.0)
prob = ODEProblem(lorenz63!, zeros(Nx), tspan)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 100.0)
u0: 3-element Vector{Float64}:
 0.0
 0.0
 0.0

Set initial condition of the true system

In [15]:
x0 = rand(model.π0);

In [16]:
data = generate_lorenz63(model, x0, Tf);

Initialize the ensemble matrix `X` $\in \mathbb{R}^{(N_y + N_x) \times N_e}$.

In [17]:
# Ensemble size
Ne = 300

X0 = zeros(model.Ny + model.Nx, Ne)

# Generate the initial conditions for the state.
viewstate(X0, model.Ny, model.Nx) .= rand(model.π0, Ne)

3×300 view(::Matrix{Float64}, 4:6, :) with eltype Float64:
  1.10411   -1.15526    0.161436  …   0.31848    0.883481   1.23124
 -0.192084  -1.17398   -0.780307     -0.988889  -0.700049  -0.585068
  1.34883    0.833289  -1.46338       0.179696   0.928431   0.0801317

Use the stochastic ensemble Kalman filter for the spin-up phase. There is no reason to use the stochastic map filter over the first cycles, as the performance of the inference is determined by the quality of the ensemble, not the quality of the filter.

In [18]:
enkf = StochEnKF(x->x, ϵy, Δtdyn, Δtobs)

Stochastic EnKF  with filtered = false


In [19]:
Xenkf = seqassim(F, data, Tf, model.ϵx, enkf, deepcopy(X0), model.Ny, model.Nx, t0);

In [20]:
tspin = 50.0
Tspin = ceil(Int64, (tspin-t0)/Δtobs)

250

Time average root-mean-squared error 

In [21]:
rmse_enkf = mean(map(i->norm(data.xt[:,i]-mean(Xenkf[i+1]; dims = 2))/sqrt(Nx), Tspin:Tf))

0.7919033201181368

Initialize the structure of the map

In [22]:
Xspin = vcat(zeros(Ny, Ne), deepcopy(Xenkf[Tspin+1]))

6×300 Matrix{Float64}:
   0.0       0.0       0.0       0.0     …    0.0       0.0       0.0
   0.0       0.0       0.0       0.0          0.0       0.0       0.0
   0.0       0.0       0.0       0.0          0.0       0.0       0.0
 -10.327   -10.7509  -10.8743  -10.7626     -10.492   -10.7172  -10.9578
 -14.6299  -15.4301  -15.2748  -15.2683     -15.0873  -15.1435  -15.5442
  22.6578   23.7576   23.9253   23.9832  …   23.736    23.8891   23.6032

In [23]:
# Validation of the setup

Xspin = vcat(zeros(Ny, Ne), deepcopy(Xenkf[Tspin+1]))

ystar = data.yt[:,250]
data.tt[250]

ϵx(Xspin, Ny+1, Ny+Nx)

# Compute measurements
observe(F.h, Xspin, tspin, Ny, Nx)

# Perturbation of the measurements
smf.ϵy(Xspin, 1, Ny)
@show norm(Xspin)

# Optimize the map S
optimize(smf.S, Xspin; start = Ny+1)

LoadError: UndefVarError: `smf` not defined

In [24]:
# Evaluate the transport map
Sx = evaluate(smf.S, Xspin; start = Ny+1)

Xpost = deepcopy(Xspin)
inverse!(Xpost, deepcopy(Sx), smf.S, ystar; start = Ny+1)

@show norm(Xpost[1:Ny,:] .- ystar)

@show norm(evaluate(smf.S, Xspin; start = Ny+1)-evaluate(smf.S, Xpost; start = Ny+1))

LoadError: UndefVarError: `smf` not defined

In [25]:
tsmf = 100.0
Tsmf = ceil(Int64, (tsmf-tspin)/Δtobs)

250

In [26]:
p = 2
# order = [[-1], [1; 1], [-1; 1; 0], [-1; 1; 1; 0]]
order = [[-1], [p; p], [-1; p; 0], [-1; p; p; 0]]

# order = [[-1], [-1; -1], [-1; -1; -1], [p; -1; -1 ;p], [-1; p; -1; p; p], [-1; -1; p; p; p; p]]

# parameters of the radial map
γ = 2.0
λ = 0.0
δ = 1e-8
κ = 10.0
β = 1.0

dist = Float64.(metric_lorenz(3))
idx = vcat(collect(1:Ny)',collect(1:Ny)')

smf = SparseRadialSMF(x->x, F.h, β, ϵy, order, γ, λ, δ, κ, 
                      Ny, Nx, Ne, 
                      Δtdyn, Δtobs, 
                      dist, idx; islocalized = true)

SparseRadialSMF(var"#5#6"(), h, MultiplicativeInflation(1.0), AdditiveInflation(3, [0.0, 0.0, 0.0], [4.0 0.0 0.0; 0.0 4.0 0.0; 0.0 0.0 4.0], [2.0 0.0 0.0; 0.0 2.0 0.0; 0.0 0.0 2.0]), Sparse Radial Map of dimension Nx = 4 and order p = [[-1], [2, 2], [-1, 2, 0], [-1, 2, 2, 0]]
        with parameters (γ, λ, δ, κ) = (2.0, 0.0, 1.0e-8, 10.0)
, 3, 3, 0.05, 0.2, [0.0 1.0 1.0; 1.0 0.0 1.0; 1.0 1.0 0.0], [1 2 3; 1 2 3], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], false, true)

In [27]:
# Ensemble size
Ne = 1000

X0 = zeros(model.Ny + model.Nx, Ne)

# Generate the initial conditions for the state.
viewstate(X0, model.Ny, model.Nx) .= rand(model.π0, Ne)

3×1000 view(::Matrix{Float64}, 4:6, :) with eltype Float64:
 0.325792   -2.07147   0.778709  …  -0.732486   0.0351791   1.38539
 0.0152669  -0.14387   0.339421      0.339787  -1.55939     0.784483
 0.296231    0.483767  1.01657       0.753127  -2.10141    -0.447518

In [28]:
Xsmf = seqassim(F, data, Tf, model.ϵx, smf, X0, model.Ny, model.Nx, 0.0);

LoadError: DimensionMismatch: array could not be broadcast to match destination

In [29]:
rmse_smf = mean(map(i->norm(data.xt[:,i]-mean(Xsmf[i+1]; dims = 2))/sqrt(Nx), Tspin:Tf))

LoadError: UndefVarError: `Xsmf` not defined

In [30]:
(rmse_enkf-rmse_smf)/rmse_enkf

LoadError: UndefVarError: `rmse_smf` not defined

In [31]:
nb = 1
ne = Tf
Δ = 5
plt = plot(layout = grid(3,1), xlim = (-Inf, Inf), ylim = (-Inf, Inf), xlabel = L"t", 
           size = (600, 800))

for i =1:3
    plot!(plt[i,1], data.tt[nb:Δ:ne], data.xt[i,nb:Δ:ne], linewidth =  3, color = :teal, 
          ylabel = latexstring("x_"*string(i)),
         legend = (i == 1), label = "True")
    plot!(plt[i,1], data.tt[nb:Δ:ne], mean_hist(Xsmf)[i,1+nb:Δ:1+ne], linewidth = 3, grid = false,
          color = :orangered2, linestyle = :dash, label = "Stochastic Map filter")
    scatter!(plt[i,1], data.tt[nb:Δ:ne], data.yt[i,nb:Δ:ne], linewidth = 3, color = :grey, 
          markersize = 5, alpha = 0.5, label  = "Observation")
end

plt

LoadError: LoadError: UndefVarError: `@L_str` not defined
in expression starting at In[31]:4